In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20201018.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,10/14/2020 10:19:39,Fantastic Planet (1973),Walkabout (1971),The River Wild (1994),Dersu Uzala (1975),Hell in the Pacific (1968),Shin Godzilla (2016),TICKLING THE IVORIES,IRON CURTAIN,THE ARTIST IS AN ASSHOLE,HERZOGAPOOLZA,ROMANCE IS DEAD: GOTHS ON FILM,THE MOST DANGEROUS GAME,NaN
1,10/14/2020 10:46:52,Dersu Uzala (1975),Walkabout (1971),Fantastic Planet (1973),Hell in the Pacific (1968),The River Wild (1994),NaN,HERZOGAPOOLZA,IRON CURTAIN,THE ARTIST IS AN ASSHOLE,TICKLING THE IVORIES,THE MOST DANGEROUS GAME,ROMANCE IS DEAD: GOTHS ON FILM,No Theme/Free-for-all
2,10/14/2020 14:50:42,The River Wild (1994),Hell in the Pacific (1968),Dersu Uzala (1975),Fantastic Planet (1973),Shin Godzilla (2016),Walkabout (1971),ROMANCE IS DEAD: GOTHS ON FILM,THE MOST DANGEROUS GAME,IRON CURTAIN,TICKLING THE IVORIES,HERZOGAPOOLZA,THE ARTIST IS AN ASSHOLE,No Theme/Free-for-all
3,10/14/2020 14:54:33,Shin Godzilla (2016),The River Wild (1994),Hell in the Pacific (1968),Dersu Uzala (1975),Fantastic Planet (1973),NaN,IRON CURTAIN,THE MOST DANGEROUS GAME,ROMANCE IS DEAD: GOTHS ON FILM,HERZOGAPOOLZA,THE ARTIST IS AN ASSHOLE,TICKLING THE IVORIES,NaN
4,10/14/2020 16:34:43,Dersu Uzala (1975),Shin Godzilla (2016),Hell in the Pacific (1968),Walkabout (1971),The River Wild (1994),Fantastic Planet (1973),THE ARTIST IS AN ASSHOLE,HERZOGAPOOLZA,THE MOST DANGEROUS GAME,ROMANCE IS DEAD: GOTHS ON FILM,IRON CURTAIN,TICKLING THE IVORIES,No Theme/Free-for-all
5,10/14/2020 19:51:07,Shin Godzilla (2016),Hell in the Pacific (1968),Walkabout (1971),Dersu Uzala (1975),The River Wild (1994),Fantastic Planet (1973),ROMANCE IS DEAD: GOTHS ON FILM,THE ARTIST IS AN ASSHOLE,TICKLING THE IVORIES,IRON CURTAIN,THE MOST DANGEROUS GAME,HERZOGAPOOLZA,No Theme/Free-for-all
6,10/14/2020 19:53:16,Shin Godzilla (2016),Walkabout (1971),The River Wild (1994),Dersu Uzala (1975),Hell in the Pacific (1968),Fantastic Planet (1973),THE MOST DANGEROUS GAME,THE ARTIST IS AN ASSHOLE,IRON CURTAIN,HERZOGAPOOLZA,ROMANCE IS DEAD: GOTHS ON FILM,TICKLING THE IVORIES,No Theme/Free-for-all
7,10/15/2020 14:52:13,Shin Godzilla (2016),Walkabout (1971),The River Wild (1994),Hell in the Pacific (1968),Dersu Uzala (1975),Fantastic Planet (1973),HERZOGAPOOLZA,IRON CURTAIN,THE MOST DANGEROUS GAME,ROMANCE IS DEAD: GOTHS ON FILM,THE ARTIST IS AN ASSHOLE,TICKLING THE IVORIES,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Fantastic Planet (1973)', 'Walkabout (1971)',
       'The River Wild (1994)', 'Dersu Uzala (1975)',
       'Hell in the Pacific (1968)', 'Shin Godzilla (2016)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Fantastic Planet (1973)': <Candidate('Fantastic Planet (1973)')>,
 'Walkabout (1971)': <Candidate('Walkabout (1971)')>,
 'The River Wild (1994)': <Candidate('The River Wild (1994)')>,
 'Dersu Uzala (1975)': <Candidate('Dersu Uzala (1975)')>,
 'Hell in the Pacific (1968)': <Candidate('Hell in the Pacific (1968)')>,
 'Shin Godzilla (2016)': <Candidate('Shin Godzilla (2016)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Fantastic Planet (1973), Walkabout (1971), The River Wild (1994), Dersu Uzala (1975), Hell in the Pacific (1968), Shin Godzilla (2016))>,
 <Ballot(Dersu Uzala (1975), Walkabout (1971), Fantastic Planet (1973), Hell in the Pacific (1968), The River Wild (1994))>,
 <Ballot(The River Wild (1994), Hell in the Pacific (1968), Dersu Uzala (1975), Fantastic Planet (1973), Shin Godzilla (2016), Walkabout (1971))>,
 <Ballot(Shin Godzilla (2016), The River Wild (1994), Hell in the Pacific (1968), Dersu Uzala (1975), Fantastic Planet (1973))>,
 <Ballot(Dersu Uzala (1975), Shin Godzilla (2016), Hell in the Pacific (1968), Walkabout (1971), The River Wild (1994), Fantastic Planet (1973))>,
 <Ballot(Shin Godzilla (2016), Hell in the Pacific (1968), Walkabout (1971), Dersu Uzala (1975), The River Wild (1994), Fantastic Planet (1973))>,
 <Ballot(Shin Godzilla (2016), Walkabout (1971), The River Wild (1994), Dersu Uzala (1975), Hell in the Pacific (1968), Fantastic Planet (1973))>,
 <Ballot(Sh

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                     Votes  Status
--------------------------  -------  --------
Shin Godzilla (2016)              4  Elected
Dersu Uzala (1975)                2  Rejected
The River Wild (1994)             1  Rejected
Fantastic Planet (1973)           1  Rejected
Walkabout (1971)                  0  Rejected
Hell in the Pacific (1968)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                         Votes  Status
------------------------------  -------  --------
HERZOGAPOOLZA                         2  Hopeful
ROMANCE IS DEAD: GOTHS ON FILM        2  Hopeful
IRON CURTAIN                          1  Hopeful
THE ARTIST IS AN ASSHOLE              1  Hopeful
THE MOST DANGEROUS GAME               1  Hopeful
TICKLING THE IVORIES                  1  Rejected
No Theme/Free-for-all                 0  Rejected

ROUND 2
Candidate                         Votes  Status
------------------------------  -------  --------
IRON CURTAIN                          2  Hopeful
HERZOGAPOOLZA                         2  Hopeful
ROMANCE IS DEAD: GOTHS ON FILM        2  Hopeful
THE ARTIST IS AN ASSHOLE              1  Rejected
THE MOST DANGEROUS GAME               1  Rejected
TICKLING THE IVORIES                  0  Rejected
No Theme/Free-for-all                 0  Rejected

ROUND 3
Candidate                         Votes  Status
------------------------------  ------